# 스팀게임목록 정제

## 조건
- 대상 : 크롤링된 Steam 멀티플레이어 카테고리 게임 목록 데이터
- 중복 제거 : '게임명' 기준 첫 번째 행만 유지
- DLC여부 생성
    - 다른 행에 해당 게임명의 DLC가 존재하면 '1'
    - 그렇지 않으면 '0'
- '게임명'에 'DLC'가 포함된 행은 삭제
- 부가상품 제외:
  - 키워드: 'pack', '팩', '세트', '패스' 등
  - 패턴: '" - "'(공백-하이픈-공백)
- 출력: '01-2. steam_data.csv' (DLC와 부가상품 제거 후 원게임만 남김)

## 절차
1. 원본 CSV 로드 후 게임명 기준 중복 제거
2. '게임명'에 'DLC' 포함된 행을 찾아 원게임명 추출
3. 원게임별로 DLC여부 열을 생성하여 DLC 존재 여부 입력
4. '게임명'에 'DLC' 포함된 행 삭제
5. 부가상품 필터링:
   - 'pack', '팩', '세트', '패스' 등 포함 행 제거
   - '" - "' 패턴 포함 행 제거
6. 결과 저장: '02-1. steam_data_final.csv'

## 데이터프레임
- df : 스팀 멀티플레이어 목록
- df_data : DLC 여부 추가 후 원래 게임만 남긴 목록

### 1. DLC여부 열 만들기

In [ ]:
import pandas as pd
df = pd.read_csv("01-1. steam_multiplayer_combined.csv", encoding="utf-8-sig")

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45934 entries, 0 to 45933
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   게임명       45933 non-null  object
 1   출시일       45817 non-null  object
 2   장르        45933 non-null  object
 3   OS        45930 non-null  object
 4   평가점수      20776 non-null  object
 5   사용자 평가 수  20776 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


In [21]:
df = df.drop_duplicates(subset="게임명", keep="first")

In [ ]:
import pandas as pd
import re

# 1. DLC가 포함된 행 찾기
dlc_rows = df["게임명"].fillna("").str.lower().str.contains("dlc")
dlc_names = df.loc[dlc_rows, "게임명"]

# 2. DLC에서 원게임명 추출하는 함수
def extract_base_game_name(dlc_name):
    """DLC 이름에서 원게임명을 추출"""
    if pd.isna(dlc_name):
        return ""
    
    # 일반적인 DLC 패턴들
    patterns = [
        r'^(.+?)\s*-\s*DLC',  # "Game Name - DLC"
        r'^(.+?)\s*:\s*DLC',  # "Game Name : DLC"
        r'^(.+?)\s+DLC',      # "Game Name DLC"
        r'^(.+?)\s*-\s*.+DLC', # "Game Name - Something DLC"
        r'^(.+?)\s*:\s*.+DLC', # "Game Name : Something DLC"
    ]
    
    for pattern in patterns:
        match = re.search(pattern, dlc_name, re.IGNORECASE)
        if match:
            return match.group(1).strip()
    
    # 패턴이 맞지 않으면 "DLC" 단어 제거 후 반환
    return re.sub(r'\s*dlc.*', '', dlc_name, flags=re.IGNORECASE).strip()

# 3. DLC별로 원게임명 추출
dlc_to_base = {}
for dlc_name in dlc_names:
    base_name = extract_base_game_name(dlc_name)
    if base_name:
        dlc_to_base[dlc_name] = base_name

# 4. 각 게임에 대해 DLC가 있는지 확인하여 DLC여부 열 생성
def check_has_dlc(game_name):
    """해당 게임명이 DLC를 가지고 있는지 확인"""
    if pd.isna(game_name):
        return 0
    
    game_name = str(game_name).strip()
    
    # 자기 자신이 DLC면 0 (DLC는 제외)
    if "dlc" in game_name.lower():
        return 0
    
    # 다른 DLC들의 원게임명과 일치하는지 확인
    for dlc_name, base_name in dlc_to_base.items():
        if game_name == base_name:
            return 1
    
    return 0

# 5. 기존 df에 "DLC여부" 열 추가
df["DLC여부"] = df["게임명"].apply(check_has_dlc)

# 6. DLC 행들 제거 (DLC가 아닌 원게임들만 남김)
df_filtered = df[~dlc_rows].copy()

# 7. 결과 출력
print("=== 원본 데이터에 DLC여부 열 추가됨 ===")
print(f"전체 행 수: {len(df)}")
print(f"DLC 제거 후: {len(df_filtered)}")
print(f"DLC가 있는 게임: {df_filtered['DLC여부'].sum()}개")
print(f"DLC가 없는 게임: {len(df_filtered) - df_filtered['DLC여부'].sum()}개")

print("\n=== 결과 샘플 ===")
print(df_filtered[["게임명", "DLC여부"]].head(10))

=== 원본 데이터에 DLC여부 열 추가됨 ===
전체 행 수: 41979
DLC 제거 후: 39941
DLC가 있는 게임: 35개
DLC가 없는 게임: 39906개

=== 결과 샘플 ===
                                                 게임명  DLC여부
0                              Football Manager 2024      0
1                                             RIDE 5      0
2  Assetto Corsa Competizione - 24H Nürburgring Pack      0
3                                       Broken Arrow      0
4                               GUILTY GEAR -STRIVE-      0
5                         Magic: The Gathering Arena      0
6                                   EZ2ON REBOOT : R      0
7                                    Automobilista 2      0
8                                 Sons Of The Forest      0
9                                      Left 4 Dead 2      0

=== DLC 매칭 결과 (확인용) ===
DLC: Need for Speed™ Most Wanted 컴플리트 DLC 번들 → 원게임: Need for Speed™ Most Wanted 컴플리트
DLC: Baldur's Gate 3 - Digital Deluxe Edition DLC → 원게임: Baldur's Gate 3 - Digital Deluxe Edition
DLC: EZ2ON REBOOT : R - 

In [31]:
df_filtered['게임명'].str.contains("DLC", case=False).sum()

0

In [ ]:
df_filtered.to_csv("01-1. steam_data.csv", index=False, encoding="utf-8-sig")

### 2. 팩, 세트 등등 부가적인거 빼고 원 게임만 남기기

In [ ]:
df_data = pd.read_csv("01-1. steam_data.csv", encoding="utf-8-sig")

In [50]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39941 entries, 0 to 39940
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   게임명       39940 non-null  object
 1   출시일       39834 non-null  object
 2   장르        39940 non-null  object
 3   OS        39938 non-null  object
 4   평가점수      18041 non-null  object
 5   사용자 평가 수  18041 non-null  object
 6   DLC여부     39941 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


In [ ]:
pattern = r"(pack|\s팩|세트|\s패스|\s-\s)"
df_data = df_data[~df_data["게임명"].str.contains(pattern, flags=re.IGNORECASE, na=False)]

In [62]:
df_data

,게임명,출시일,장르,OS,평가점수,사용자 평가 수,DLC여부
0,Football Manager 2024,2023년 11월 7일,시뮬레이션 | 스포츠 | 경영 | 축구 | 전략,Windows,평가 점수: 매우 긍정적,"사용자 평가 14,753개",0
1,RIDE 5,2023년 8월 24일,레이싱 | 시뮬레이션 | 오토바이 | 스포츠 | 분할 화면,Windows,평가 점수: 매우 긍정적,"사용자 평가 2,092개",0
3,Broken Arrow,2025년 6월 20일,전략 | 전쟁 게임 | 현대 | 실시간 전략 | 실시간 전술,Windows,평가 점수: 복합적,"사용자 평가 12,836개",0
4,GUILTY GEAR -STRIVE-,2021년 6월 11일,격투 | 2D 격투 | 웅장한 사운드트랙 | 애니메이션 | PvP,Windows,평가 점수: 매우 긍정적,"사용자 평가 43,066개",0
5,Magic: The Gathering Arena,2023년 5월 24일,전략 | 카드 게임 | 트레이딩 카드 게임 | 무료 플레이 | 덱빌딩,Windows,평가 점수: 복합적,"사용자 평가 21,004개",0
...,...,...,...,...,...,...,...
39936,A Wizard's Wool,발표 예정,액션 | 어드벤처 | 액션 어드벤처 | 플랫폼 | 퍼즐,Windows,NaN,NaN,0
39937,WATERTRIX ONLINE,출시 예정,시뮬레이션 | 캐주얼 | e스포츠 | 2D 플랫폼 | 퍼즐 플랫폼,Windows,NaN,NaN,0
39938,Rage Ball,출시 예정,3D 플랫폼 | 캐주얼 | 플랫폼 | 3D | 물리,Windows,NaN,NaN,0
39939,Trolling Bowling,출시 예정,볼링 | 멀티플레이어 | 파티 게임 | 현실적 | 캐주얼,Windows,NaN,NaN,0


In [63]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23927 entries, 0 to 39940
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   게임명       23926 non-null  object
 1   출시일       23826 non-null  object
 2   장르        23926 non-null  object
 3   OS        23925 non-null  object
 4   평가점수      13230 non-null  object
 5   사용자 평가 수  13230 non-null  object
 6   DLC여부     23927 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


In [ ]:
df_data.to_csv("01-2. steam_data_final.csv", index=False, encoding="utf-8-sig")